insert 목표: 문맥에 크게 영향을 주지 않는 선에서 단어들을 추가함.<br>
즉, 다른 말로 하면 문맥을 유지한다는 말임.
어떻게 보면 redundant한 단어들이 될 수 있지만, 이것 또한 LSTM에게 generalization 잘 할 수 있도록 해주는 동기가 되지 않을까? <br>

바꾸는 문제 (교체)는 중의성 문제를 잘 해결해야 하지만,
추가하는 문제는 중의성 문제가 크리티컬 하지 않다. (구문적 중의성만 해결할 수 있어도 괜찮지 않을까?)



## Collocation이란?

## Oxford Collocations

추가할 수 있는 품사 : 부사, 형용사
* 동사 기준으로 collocation 부사를 추가
* 형용사 기준으로 collocation 부사를 추가
* 명사 기준으로 collocation 형용사를 추가

collocation 부사/형용사 list 중에 하나를 선택하는 일 역시 중의성 문제를 푸는 일이다. 어떤 기준을 가지겠는가? 우선 아무거나 선택해도 큰 문제는 없을 것이다. 의미가 많이 다르면, Oxford Collocation 사전에서는 번호를 매겨 구분지어 놓았다. 만약 번호로 구분지어 놓으면 해당 단어는 패스하자. 의미가 크게 다르지 않으면 단어면 '|'으로만 구분지어놓았는데 얘네들은 구분짓지 말고 리스트로 모아 랜덤으로 하나 선택해도 괜찮을 듯 하다. 그래도 선택할 수 있는 한 가지 척도가 있다면, word embedding의 similarity를 활용할 수도 있겠다. 완전히 의미적 중의성은 해결해주지 못하나 빈도수가 높은 애들은 선택할 수 있다는 데에 의의를 둘 수 있다.

## 나중에 sense2vec과 google n-gram도 고려해보자.

sense2vec의 결과가 phrase로 나오는게 많은데, 활용해보자.
즉, 형용사를 insert했다고 볼 수 있다.

내가 그냥 collocation verb-adverb 사전을 만들지 <br>
아니면, nltk collocation (bigram, trigram)을 사용할지는 좀 더 고민해보자. 

In [1]:
# import nltk
# from nltk.collocations import *
# from nltk.tokenize import word_tokenize

# text = "this is a foo bar bar black sheep  foo bar bar black sheep foo bar bar black  sheep shep bar bar black sentence"

# trigram_measures = nltk.collocations.TrigramAssocMeasures()
# finder = TrigramCollocationFinder.from_words(word_tokenize(text))

# for i in finder.score_ngrams(trigram_measures.pmi):
#     print(i)

In [2]:
import spacy
import numpy
#import sense2vec
import random
import pickle

from numpy import dot
from numpy.linalg import norm
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from data_handling_for_heuristic import *

stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
nlp = spacy.load('en')
#model = sense2vec.load()

# Load Dataset

In [ ]:
# #read_path = 'data/conll2003/eng.train'
# read_path = 'train.txt'
# with open(read_path, "r") as ins:
#     raw_data = []
#     label_data = []
#     temp_sent = ''
#     temp_label = ''
#     tkn = False
#     for line in ins:
#         #array.append(line)
#         if len(line) == 1:
#             raw_data.append(temp_sent)
#             label_data.append(temp_label)
#             temp_sent = ''
#             temp_label = ''
#             tkn = False
#         else:
#             if tkn == True:
#                 temp_sent += ' '
#                 temp_label += ' '
#             temp_sent += line.split()[0] # 단어
#             temp_label += line.split()[-1] # NER 라벨
#             tkn = True
            
# sent_raw = []
# sent_label = []
# cnt = 0
# for i, row in enumerate(raw_data):
#     row_nlp = nlp(row)

# #     if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
# #         temp = []
# #         for token in row_nlp:
# #             temp.append(token.pos_) # 각 token의 pos 저장
# #         if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
# #             if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
# #                 sent_raw.append(raw_data[i])
# #                 sent_label.append(label_data[i]) 
 
    
    
#     if raw_data[i][-1] == '.' or raw_data[i][-1] == '"': # 마지막에 쉼표가 있는 문장들만 선별하자.
#         if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
#             temp = []
#             for token in row_nlp:
#                 temp.append(token.pos_) # 각 token의 pos 저장
#             if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
#                 if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
#                     sent_raw.append(raw_data[i])
#                     sent_label.append(label_data[i]) 
# #                 else:
# #                     cnt += 1


In [6]:
read_file_path = '../data/conll2003/en/train.txt'
raw_data, label_data = load_conll2003(read_file_path)

filter_none_entity = True
if filter_none_entity == True:
    sent_raw, sent_label = filtering_none_entity(raw_data, label_data)
else:
    sent_raw = raw_data[:]
    sent_label = label_data[:]

[filtering_none_entity]: before = 14987 and after = 11132


# Load Oxford Collocation Dictionary

In [7]:
def load(name):
    filehandler = open(name, "rb")
    return pickle.load(filehandler)
friend_list = load('resources/for_insert/friend_list')
voca = load('../resources/for_insert/voca')

# Main

In [8]:
n_candidates = 10

In [9]:
def similarity(self, other):
    self = nlp.vocab[self]
    other = nlp.vocab[other]
    if self.vector_norm == 0 or other.vector_norm == 0:
        return 0.0
    return numpy.dot(self.vector, other.vector) / (self.vector_norm * other.vector_norm)

import numpy as np
def max_friend(friend_list, token):
    sim_score = [0] * len(friend_list)
    for i, each in enumerate(friend_list):
        sim_score[i] = similarity(token, friend_list[i])
    
    sim_score.sort(key=lambda x: x, reverse=True)
    sim_score = sim_score[:n_candidates]
    sim_score = list(filter(lambda x: x!= 0.0, sim_score))
    if len(sim_score)==0:
        return 'none'
    
    #print(sim_score)
    #print('\n')
    #print(np.array(sim_score).max())

    choiced_score = random.choice(sim_score)
    #print(choiced_score)
    
    #print(np.array(sim_score).max(), '--', choiced_score)
    
    max1_index = np.where(sim_score == np.array(sim_score).max())[0][0]
    max_index = np.where(sim_score == choiced_score)[0][0]
    
    #print(max1_index, max_index)
    #print('\n')

    return friend_list[max_index]

def insert_module(token, pos):
    #print('------>',token, pos)
    try:voca_index = voca.index((token, pos))
    except(ValueError):
        #print('*********>', 'none')
        return 'none'
    
    if len(friend_list[voca_index])==0:
        #print('*********>', 'none')
        return 'none'
    else:
        new_token = max_friend(friend_list[voca_index], token)
        #print('*********>',new_token)
        return new_token

In [10]:
def window_filtering(sent_label, window_size=3):
    list_sent_label = sent_label.split()
    new_list = [0] * len(list_sent_label)
    ### 순방향
    for i, _ in enumerate(list_sent_label):
        temp = list_sent_label[i:i+1+window_size]
        #print(temp)
        tkn = False
        for each in temp:
            if each != 'O':
                tkn = True
        if tkn == True:
            new_list[i] = 1
    ### 역방향
    for i  in range(len(list_sent_label)-1, -1, -1):
        idx = i-window_size
        if idx < 0:
            idx = 0
        temp = list_sent_label[idx:i]
        tkn = False
        for each in temp:
            if each != 'O':
                tkn = True
        if tkn == True:
            new_list[i] = 1      
    return new_list  

In [21]:
import nltk
insert_sent_cnt = 0
new_sent_list = []
new_label_list = [] 



for i, _ in enumerate(sent_raw):
    
    #list_sent_raw = sent_raw[i].split()
    str_sent_raw = preprocessing_for_spacy(sent_raw[i])
    list_sent_nlp = nlp(str_sent_raw)
    
    list_sent_raw = str_sent_raw.split()
    list_sent_label = sent_label[i].split()
    
    #print(str_sent_raw)
    
    ###################################
    """ 조건에 맞는 token들을 불러온다 """
    ###################################
    pre_token_pos = 'none'
    insert_on = False
    add_idx = 0
    
    ### filtering with window size based on entities
    filtered_list = window_filtering(sent_label[i])
        
    if len(list_sent_nlp) != len(list_sent_label):
        print('[ERROR]: len(list_sent_nlp) != len(list_sent_label)')
        continue
    
    for i, token in enumerate(list_sent_nlp):
        
        new_token = 'none' # 계속 'none'으로 남아있다면 추가하지 않을 것임.
    
        # 엔티티는 건드리지 말자.
        
        if list_sent_label[i+add_idx] != 'O':
            continue
        if filtered_list[i+add_idx] == 0: # filtering with window size based on entities
            continue
        
        
        
        
        #print('>>>', token, token.pos_, pre_token_pos)
        #################
        ### INSERT_MODULE  
        str_token = str(token).lower()
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN': # 오로지 하나의 명사일경우...
            if i==0:
                str_token = lemmatiser.lemmatize(str_token, 'n')
                new_token = insert_module(str_token, 'noun')
            else:
                if pre_token_pos != 'ADJ':
                    if pre_token_pos != 'NOUN':
                        if pre_token_pos != 'PROPN':
                            str_token = lemmatiser.lemmatize(str_token, 'n')
                            new_token = insert_module(str_token, 'noun')

        elif token.pos_ == 'VERB': 
            str_token = lemmatiser.lemmatize(str_token, 'v')
            new_token = insert_module(str_token, 'verb')

        elif token.pos_ == 'ADJ':
            new_token = insert_module(str_token, 'adj')
        
        ################
        ### REAL INSERT
        #print('new token ---> ', new_token)
        
        pre_token_pos = token.pos_
        
        if new_token == 'none': # insert된 것이 하나도 없으면 pass.
            continue # pass
        else:
            insert_on = True # insert 한 번 이상 되었음.
            list_sent_raw.insert(i+add_idx, new_token)
            list_sent_label.insert(i+add_idx, 'O')
            filtered_list.insert(i+add_idx, -1)
            add_idx += 1
        
        #print('\n')
        
    
    #print(' '.join(list_sent_raw))
    #print(' '.join(list_sent_label))
    #print('\n')
    ### Append new sent list
    if insert_on == True:
        insert_sent_cnt += 1
        new_sent_list.append(list_sent_raw)
        new_label_list.append(list_sent_label)
    
    #break

[ERROR]: len(list_sent_nlp) != len(list_sent_label)
[ERROR]: len(list_sent_nlp) != len(list_sent_label)
[ERROR]: len(list_sent_nlp) != len(list_sent_label)


KeyboardInterrupt: 

# Store to file

In [ ]:
len(new_sent_list)

In [ ]:
# path_write = 'data_logic_warehouse/type_2/insert_'+str(len(new_sent_list))+'.txt'

# with open(path_write, 'w', encoding='UTF-8') as txt:
#     for i, new_token_list in enumerate(new_sent_list):
#         for j, token in enumerate(new_token_list):
#             txt.write(new_sent_list[i][j]+' '+'NNP'+' '+'B-NP'+' '+new_label_list[i][j])
#             txt.write('\n')
#         txt.write('\n')

### 예외처리

a bit와 같이 두 글자로 이뤄진 것을 예외처리 할 필요가 있다. 여기서 앞 글자만 따 similarity를 측정하면 높게 나온다. a는 어디서나 자주 등장하기 때문에... 따라서 외자는 제외한다는 식으로 예외처리가 필요하다.